## Парсер заданий с текстом Решу ЕГЭ/ОГЭ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Устанавливаем selenium webdriver в Colab

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
clear_output()

In [ ]:
import sys
from selenium import webdriver
import time
from tqdm.notebook import tqdm
import re
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import pickle

In [ ]:
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    return browser

In [ ]:
browser = get_browser()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# ссылки на подборки заданий
links = ["https://rus-ege.sdamgia.ru/test?theme=330",
         "https://rus-ege.sdamgia.ru/test?theme=323",
         "https://rus-ege.sdamgia.ru/test?theme=240",
         "https://rus-ege.sdamgia.ru/test?theme=332",
         "https://rus-ege.sdamgia.ru/test?theme=331",
         
         "https://rus-oge.sdamgia.ru/test?theme=128",
         "https://rus-oge.sdamgia.ru/test?theme=114",
         "https://rus-oge.sdamgia.ru/test?theme=99"]

In [ ]:
dataset = set()
NUM = 32 # как долго будем крутить страницу вниз, для получения заданий
TIMEOUT = 0.2 # сколько будем ждать загрузку новых заданий

for link in tqdm(links):
    browser.get(link)
    for _ in tqdm(range(NUM)):
        html = browser.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        time.sleep(TIMEOUT)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        texts = soup.find_all("div", {"class": "probtext"})
        for text in texts:
            p = text.find_all("p") #, {"class":"left_margin"}
            paragraphs = []
            for line in p:
                try:
                    if re.search(r"\(\d+\)", line.text).span():
                        paragraphs.append(line.text)
                except: pass
            if paragraphs:
                dataset.add((link, tuple(paragraphs)))

In [ ]:
# для просмотра экрана браузера
browser.save_screenshot(f'screenshots.png')

True

In [ ]:
len(dataset) # сколько заданий собрали

339

In [ ]:
with open(f"drive/MyDrive/sdamgia_{len(dataset)}.pkl", "wb") as f:
    pickle.dump(dataset, f)